In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uom190346a/sleep-health-and-lifestyle-dataset")

print("Path to dataset files:", path)

#https://www.kaggle.com/code/jillanisofttech/sleep-health-and-lifestyle-predication-with-94-ac
#eksempel av noen sin løsning på dette datasetet

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv(path+"/Sleep_health_and_lifestyle_dataset.csv")

In [ ]:
headAmount = 5 # for å skrive ut alle data punkt i csv file skriv data.size() istedenfor eit tall
data.head(headAmount)

In [ ]:
print(data['Sleep Disorder'].unique())
print(data['Occupation'].unique())
print(data['BMI Category'].unique())

In [ ]:
data['Blood Pressure'].unique()

In [ ]:
# Vi deler opp blodtrykk siden det er to verdiar i ein kolonne
data_16_pro_xl = (pd.concat([data, data['Blood Pressure'].str.split('/', expand=True)], axis=1).
                  drop('Blood Pressure', axis=1))

In [ ]:
data_16_pro_xl

In [ ]:
data_16_pro_xl = data_16_pro_xl.rename(columns={0: 'Systolic Blood Pressure', 1: 'Diastolic Blood Pressure'})

In [ ]:
data_16_pro_xl

In [ ]:
data_16_pro_xl['Systolic Blood Pressure'] = data_16_pro_xl['Systolic Blood Pressure'].astype(float)
data_16_pro_xl['Diastolic Blood Pressure'] = data_16_pro_xl['Diastolic Blood Pressure'].astype(float)
data_16_pro_xl.info()

In [ ]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
data_16_pro_xl['Occupation']= label_encoder.fit_transform(data_16_pro_xl['Occupation'])
data_16_pro_xl['BMI Category']= label_encoder.fit_transform(data_16_pro_xl['BMI Category'])
data_16_pro_xl['Sleep Disorder']= label_encoder.fit_transform(data_16_pro_xl['Sleep Disorder'])
data_16_pro_xl['Gender']= label_encoder.fit_transform(data_16_pro_xl['Gender'])

In [ ]:
X = data_16_pro_xl.drop('Sleep Disorder','Person ID', axis=1)
y = data_16_pro_xl['Sleep Disorder']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

